In [10]:
import networkx as nx
import numpy as np

# create a Barabási–Albert graph
n = 1e4
m = 3
G = nx.barabasi_albert_graph(n, m)

# get the adjacency matrix
A = nx.adjacency_matrix(G)

# scale the matrix to make each column sum to 1
A = A.todense()
A = A / A.sum(axis=0)

# save the matrix in row,col,val format
with open("./data/graph.txt", "w") as f:
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i, j] != 0:
                f.write("{} {} {}\n".format(i, j, A[i, j]))

In [7]:
A = np.array([[1, 2], [3, 4]])
A.sum(axis=0)

array([4, 6])